In [1]:
import numpy as np
from copy import deepcopy
from tqdm import tqdm
import random
from typing import Any, Tuple, Optional, Sequence

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.nn import SAGEConv
from torch_geometric.loader import DataLoader
from torch.autograd import Function

/home/hhchung/dyngraph-uda/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sklearn.metrics import f1_score

In [4]:
import sys
sys.path.append("..")
from dataset import load_nc_dataset

In [5]:
import matplotlib.pyplot as plt

In [6]:
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

## Model Structure ##

In [7]:
class TwoLayerGraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        return x
    
class MLPHead(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.linear1 = nn.Linear(in_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x):
        x1 = self.linear1(x)
        x1 = F.elu(x1)
        x1 = F.dropout(x1, p=self.dropout)
        output = self.linear2(x1)
        # x = F.softmax(x, dim=1)
        features = x1
        return output, features


## Data Preparation ##

* Train: 0-6
* Val: 7, 8
* Test and adapt: 9-13, 14-18, 19-23, 24-28, 29-33, 34-38, 39-43, 44-48

In [8]:
def get_data(data_dir, dataset, sub_dataset=None):
    if dataset == 'elliptic':
        data = load_nc_dataset(data_dir, 'elliptic', sub_dataset)
    else:
        raise ValueError('Invalid dataname')
    # if len(data.y.shape) == 1:
    #     data.y = data.y.unsqueeze(1)
    return data

In [9]:
data_dir = '/home/hhchung/data/graph-data/elliptic_bitcoin_dataset'

## Train / Test / Adapt Loop ##

In [10]:
def train(encoder, mlp, optimizer, loader, loss_fn, device='cpu'):
    encoder.train()
    mlp.train()
    optimizer.zero_grad()
    
    total_train_loss = 0
    for data in loader:
        data = data.to(device)
        out, _ = mlp(encoder(data.x, data.edge_index))
        loss = loss_fn(out[data.mask], data.y[data.mask])
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    
    total_train_loss /= len(loader)
    return total_train_loss

@torch.no_grad()
def test(encoder, mlp, loader, loss_fn, device='cpu'):
    encoder.eval()
    mlp.eval()
    total_val_loss = 0
    total_f1 = 0
    for data in loader:
        data = data.to(device)
        out, _ = mlp(encoder(data.x, data.edge_index))
        loss = loss_fn(out[data.mask], data.y[data.mask])
        y_pred = torch.argmax(out, dim=1)
        f1 = f1_score(y_pred[data.mask].detach().cpu().numpy(), data.y[data.mask].detach().cpu().numpy())
        total_val_loss += loss.item()
        total_f1 += f1
    total_val_loss /= len(loader)
    total_f1 /= len(loader)
    return total_val_loss, total_f1



In [11]:
def pseudo_label(encoder, classifier, tgt_data, confidence=0.8):
    pseudo_y, _ = classifier(encoder(tgt_data.x, tgt_data.edge_index))
    pseudo_y = F.softmax(pseudo_y, dim=1)
    pseudo_y_confidence, pseudo_y_hard_label = torch.max(pseudo_y, dim=1)
    pseudo_mask = pseudo_y_confidence > confidence
    return pseudo_y_hard_label, pseudo_mask

def adapt_train(encoder, classifier, device, src_loader, tgt_loader, optimizer, lambda_coeff=1):
    encoder.train()
    classifier.train()
    
    
    total_src_loss = 0
    total_src_node = 0
    for src_data in tqdm(src_loader):
        src_data = src_data.to(device)
        src_y, _ = classifier(encoder(src_data.x, src_data.edge_index))
        total_src_loss += F.nll_loss(F.log_softmax(src_y[src_data.mask], dim=1), src_data.y[src_data.mask], reduction='sum')
        total_src_node += src_data.mask.sum().item()
    total_src_loss /= total_src_node
    
    
    total_tgt_loss = 0
    total_tgt_node = 0
    for tgt_data in tqdm(tgt_loader):
        tgt_data = tgt_data.to(device)
        pseudo_tgt_label, pseudo_tgt_mask = pseudo_label(encoder, classifier, tgt_data)
        tgt_y, _ = classifier(encoder(tgt_data.x, tgt_data.edge_index))
        total_tgt_loss += F.nll_loss(F.log_softmax(tgt_y, dim=1)[pseudo_tgt_mask], pseudo_tgt_label[pseudo_tgt_mask], reduction='sum')
        # total_tgt_node += tgt_data.x.shape[0]
        total_tgt_node += pseudo_tgt_mask.sum().item()
    total_tgt_loss /= total_tgt_node
    loss = total_src_loss + total_tgt_loss * lambda_coeff
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item(), total_src_loss.item(), total_tgt_loss.item()
    
@torch.no_grad()
def adapt_test(encoder, classifier, device, src_loader, tgt_loader, lambda_coeff=1):
    encoder.eval()
    classifier.eval()
    
    total_src_loss = 0
    total_src_node = 0
    
    for src_data in tqdm(src_loader):
        src_data = src_data.to(device)
        src_y, _ = classifier(encoder(src_data.x, src_data.edge_index))
        total_src_loss += F.nll_loss(F.log_softmax(src_y[src_data.mask], dim=1), src_data.y[src_data.mask], reduction='sum')
        total_src_node += src_data.mask.sum().item()
    total_src_loss /= total_src_node
    
    
    total_tgt_loss = 0
    total_tgt_node = 0
    for tgt_data in tqdm(tgt_loader):
        tgt_data = tgt_data.to(device)
        pseudo_tgt_label, pseudo_tgt_mask = pseudo_label(encoder, classifier, tgt_data)
        tgt_y, _ = classifier(encoder(tgt_data.x, tgt_data.edge_index))
        total_tgt_loss += F.nll_loss(F.log_softmax(tgt_y, dim=1)[pseudo_tgt_mask], pseudo_tgt_label[pseudo_tgt_mask], reduction='sum')
        # total_tgt_node += tgt_data.x.shape[0]
        total_tgt_node += pseudo_tgt_mask.sum().item()
    total_tgt_loss /= total_tgt_node
    loss = total_src_loss + total_tgt_loss * lambda_coeff
    
    return loss.item(), total_src_loss.item(), total_tgt_loss.item()



## Initial Source Stage ##

In [12]:
elliptic_0 = get_data(data_dir, 'elliptic', 0)
feat_dim = elliptic_0.x.shape[1]
hidden_dim = 128
emb_dim = 128
encoder = TwoLayerGraphSAGE(feat_dim, hidden_dim, emb_dim)
mlp = MLPHead(emb_dim, emb_dim // 4, 2)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(mlp.parameters()), lr=1e-3)
epochs = 500
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = encoder.to(device)
mlp = mlp.to(device)

../dataset.py:75: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  edge_index = torch.tensor(A.nonzero(), dtype=torch.long)


In [13]:
split = [0,7,9]
train_data = [get_data(data_dir, 'elliptic', i) for i in range(split[0],split[1])]
val_data = [get_data(data_dir, 'elliptic', i) for i in range(split[1],split[2])]
train_loader = DataLoader(dataset=train_data, batch_size=1, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=1, shuffle=False)

In [14]:
best_f1 = 0
best_encoder = None
best_mlp = None
for e in range(1, epochs + 1):
    train_loss = train(encoder, mlp, optimizer, train_loader, loss_fn, device)
    val_loss, val_f1 = test(encoder, mlp, val_loader, loss_fn, device)
    print(f"Epoch:{e}/{epochs} Train Loss:{round(train_loss,4)} Val Loss:{round(val_loss,4)} Val F1:{round(val_f1, 4)}")
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_encoder = deepcopy(encoder)
        best_mlp = deepcopy(mlp)

encoder = deepcopy(best_encoder)
mlp = deepcopy(best_mlp)

Epoch:1/500 Train Loss:0.3965 Val Loss:0.4848 Val F1:0.0039
Epoch:2/500 Train Loss:0.1371 Val Loss:0.6459 Val F1:0.004
Epoch:3/500 Train Loss:0.0979 Val Loss:0.8028 Val F1:0.0079
Epoch:4/500 Train Loss:0.1012 Val Loss:0.5666 Val F1:0.1328
Epoch:5/500 Train Loss:0.0833 Val Loss:0.3005 Val F1:0.5841
Epoch:6/500 Train Loss:0.0839 Val Loss:0.2747 Val F1:0.5738
Epoch:7/500 Train Loss:0.0655 Val Loss:0.4344 Val F1:0.1344
Epoch:8/500 Train Loss:0.0721 Val Loss:0.5049 Val F1:0.0841
Epoch:9/500 Train Loss:0.0682 Val Loss:0.3919 Val F1:0.1701
Epoch:10/500 Train Loss:0.0585 Val Loss:0.2602 Val F1:0.5697
Epoch:11/500 Train Loss:0.06 Val Loss:0.2372 Val F1:0.6526
Epoch:12/500 Train Loss:0.0514 Val Loss:0.2533 Val F1:0.589
Epoch:13/500 Train Loss:0.0478 Val Loss:0.2584 Val F1:0.5856
Epoch:14/500 Train Loss:0.0441 Val Loss:0.2189 Val F1:0.6512
Epoch:15/500 Train Loss:0.0428 Val Loss:0.2107 Val F1:0.6396
Epoch:16/500 Train Loss:0.0386 Val Loss:0.2036 Val F1:0.6899
Epoch:17/500 Train Loss:0.0388 Val Lo

Epoch:138/500 Train Loss:0.0038 Val Loss:0.239 Val F1:0.7775
Epoch:139/500 Train Loss:0.0041 Val Loss:0.249 Val F1:0.7442
Epoch:140/500 Train Loss:0.0038 Val Loss:0.2512 Val F1:0.7715
Epoch:141/500 Train Loss:0.003 Val Loss:0.271 Val F1:0.7449
Epoch:142/500 Train Loss:0.0035 Val Loss:0.2871 Val F1:0.722
Epoch:143/500 Train Loss:0.0029 Val Loss:0.2909 Val F1:0.7256
Epoch:144/500 Train Loss:0.003 Val Loss:0.3036 Val F1:0.7267
Epoch:145/500 Train Loss:0.0038 Val Loss:0.2901 Val F1:0.7193
Epoch:146/500 Train Loss:0.0028 Val Loss:0.2764 Val F1:0.7448
Epoch:147/500 Train Loss:0.0034 Val Loss:0.2884 Val F1:0.7801
Epoch:148/500 Train Loss:0.0033 Val Loss:0.3036 Val F1:0.7505
Epoch:149/500 Train Loss:0.0025 Val Loss:0.2989 Val F1:0.7433
Epoch:150/500 Train Loss:0.003 Val Loss:0.286 Val F1:0.7362
Epoch:151/500 Train Loss:0.0022 Val Loss:0.3057 Val F1:0.7445
Epoch:152/500 Train Loss:0.0026 Val Loss:0.2751 Val F1:0.7484
Epoch:153/500 Train Loss:0.0021 Val Loss:0.3205 Val F1:0.7296
Epoch:154/500 Tr

Epoch:273/500 Train Loss:0.0021 Val Loss:0.3579 Val F1:0.7599
Epoch:274/500 Train Loss:0.0015 Val Loss:0.3756 Val F1:0.7258
Epoch:275/500 Train Loss:0.0013 Val Loss:0.3763 Val F1:0.7545
Epoch:276/500 Train Loss:0.0014 Val Loss:0.3841 Val F1:0.741
Epoch:277/500 Train Loss:0.001 Val Loss:0.3823 Val F1:0.761
Epoch:278/500 Train Loss:0.0016 Val Loss:0.3814 Val F1:0.7354
Epoch:279/500 Train Loss:0.001 Val Loss:0.408 Val F1:0.738
Epoch:280/500 Train Loss:0.0011 Val Loss:0.4167 Val F1:0.7094
Epoch:281/500 Train Loss:0.0013 Val Loss:0.3942 Val F1:0.7342
Epoch:282/500 Train Loss:0.001 Val Loss:0.3799 Val F1:0.7438
Epoch:283/500 Train Loss:0.0013 Val Loss:0.4124 Val F1:0.7383
Epoch:284/500 Train Loss:0.0006 Val Loss:0.4175 Val F1:0.7486
Epoch:285/500 Train Loss:0.0008 Val Loss:0.4321 Val F1:0.7104
Epoch:286/500 Train Loss:0.0009 Val Loss:0.4261 Val F1:0.7374
Epoch:287/500 Train Loss:0.0006 Val Loss:0.3755 Val F1:0.7538
Epoch:288/500 Train Loss:0.0011 Val Loss:0.3928 Val F1:0.7582
Epoch:289/500 T

Epoch:408/500 Train Loss:0.0006 Val Loss:0.5181 Val F1:0.7514
Epoch:409/500 Train Loss:0.0012 Val Loss:0.4961 Val F1:0.7589
Epoch:410/500 Train Loss:0.0004 Val Loss:0.5068 Val F1:0.7591
Epoch:411/500 Train Loss:0.0008 Val Loss:0.5409 Val F1:0.7313
Epoch:412/500 Train Loss:0.0005 Val Loss:0.5453 Val F1:0.7247
Epoch:413/500 Train Loss:0.0006 Val Loss:0.568 Val F1:0.7514
Epoch:414/500 Train Loss:0.0016 Val Loss:0.5294 Val F1:0.7273
Epoch:415/500 Train Loss:0.0004 Val Loss:0.5211 Val F1:0.7268
Epoch:416/500 Train Loss:0.0002 Val Loss:0.5411 Val F1:0.7518
Epoch:417/500 Train Loss:0.0001 Val Loss:0.5191 Val F1:0.7584
Epoch:418/500 Train Loss:0.0004 Val Loss:0.5193 Val F1:0.7612
Epoch:419/500 Train Loss:0.0002 Val Loss:0.5352 Val F1:0.7671
Epoch:420/500 Train Loss:0.0001 Val Loss:0.5664 Val F1:0.7392
Epoch:421/500 Train Loss:0.0003 Val Loss:0.5169 Val F1:0.7306
Epoch:422/500 Train Loss:0.0003 Val Loss:0.5677 Val F1:0.7635
Epoch:423/500 Train Loss:0.0005 Val Loss:0.523 Val F1:0.7515
Epoch:424/

In [15]:
print(best_f1)

0.796399217221135


## Prequential Evaluation on Subsequent Time Steps ##

In [16]:
def continual_adapt(split, src_train_loader, src_val_loader, encoder, mlp, device, lambda_coeff=1, lr=1e-3):
    tgt_train_loader = DataLoader(dataset=[get_data(data_dir, "elliptic", i) for i in range(split[0],split[1])], batch_size=1, shuffle=True)
    tgt_val_loader = DataLoader(dataset=[get_data(data_dir, "elliptic", i) for i in range(split[1],split[2])], batch_size=1, shuffle=False)
    
    tgt_encoder, tgt_mlp = deepcopy(encoder), deepcopy(mlp)
    tgt_optimizer = torch.optim.Adam(list(tgt_encoder.parameters()) + list(tgt_mlp.parameters()), lr=lr)
    
    epochs = 500
    best_val_loss = np.inf
    best_val_src_loss, best_val_tgt_loss = None, None
    best_tgt_encoder, best_tgt_mlp = None, None
    patience = 10
    staleness = 0

    for e in range(1, epochs + 1):
        train_loss, train_src_loss, train_tgt_loss = adapt_train(tgt_encoder, tgt_mlp, device, src_train_loader, tgt_train_loader, tgt_optimizer, lambda_coeff=lambda_coeff)
        val_loss, val_src_loss, val_tgt_loss = adapt_test(tgt_encoder, tgt_mlp, device, src_val_loader, tgt_val_loader, lambda_coeff=lambda_coeff)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_src_loss = val_src_loss
            best_val_tgt_loss = val_tgt_loss
            best_tgt_encoder = deepcopy(tgt_encoder)
            best_tgt_mlp = deepcopy(tgt_mlp)
            staleness = 0
        else:
            staleness += 1
        print(f'Epoch {e}/{epochs} Train Total Loss: {round(train_loss,5)} Train Src Loss: {round(train_src_loss,5)} Train Tgt Loss: {round(train_tgt_loss,5)} \n \
                Val Total Loss: {round(val_loss,5)} Val Src Loss: {round(val_src_loss,5)} Val Tgt Loss: {round(val_tgt_loss,5)}')

        if staleness > patience:
            break

    tgt_encoder = deepcopy(best_tgt_encoder)
    tgt_mlp = deepcopy(best_tgt_mlp)
    
    return tgt_encoder, tgt_mlp, best_val_loss, best_val_src_loss, best_val_tgt_loss

In [17]:
lambda_coeff = 1
lr = 1e-4

## Finer Adapt ##
* Test: 9,10 Adapt Train: 7,8,9 Adapt Val: 10
* Test: 11,12 Adapt Train: 10,11 Adapt Val: 12
* ...
* Test: 45,46 Adapt Train: 44,45 Adapt Val: 46
* Test: 47,48

In [18]:
# test_f1_list = []
# prev_train_idx = 7
# for test_idx in range(9, 49, 2):
#     print("Test:", test_idx, test_idx + 1)
#     test_data = [get_data(data_dir, "elliptic", i) for i in range(test_idx, test_idx+2)]
#     test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)
#     test_loss, test_f1 = test(encoder, mlp, test_loader, loss_fn, device)
#     test_f1_list.append(test_f1)
#     if test_idx == 47:
#         break
#     print("Adapt train:", prev_train_idx, test_idx)
#     print("Adapt val:", test_idx + 1)
#     encoder, mlp, _, _, _ = continual_adapt([prev_train_idx, test_idx + 1, test_idx + 2], train_loader, val_loader, encoder, mlp, device, lambda_coeff=lambda_coeff, lr=lr)
#     prev_train_idx = test_idx + 1

Test: 9 10
Adapt train: 7 9
Adapt val: 10


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 191.55it/s]


Epoch 1/500 Train Total Loss: 0.08102 Train Src Loss: 0.00871 Train Tgt Loss: 0.07231 
                 Val Total Loss: 0.32434 Val Src Loss: 0.21445 Val Tgt Loss: 0.10989


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 336.78it/s]


Epoch 2/500 Train Total Loss: 0.07752 Train Src Loss: 0.00813 Train Tgt Loss: 0.0694 
                 Val Total Loss: 0.29702 Val Src Loss: 0.20191 Val Tgt Loss: 0.09511


...

Epoch 23/500 Train Total Loss: 0.04327 Train Src Loss: 0.00945 Train Tgt Loss: 0.03382 
                 Val Total Loss: 0.26223 Val Src Loss: 0.23251 Val Tgt Loss: 0.02972


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 270.36it/s]


Epoch 24/500 Train Total Loss: 0.03963 Train Src Loss: 0.00822 Train Tgt Loss: 0.03141 
                 Val Total Loss: 0.27289 Val Src Loss: 0.23844 Val Tgt Loss: 0.03444


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 154.00it/s]


Epoch 25/500 Train Total Loss: 0.04144 Train Src Loss: 0.00788 Train Tgt Loss: 0.03356 
                 Val Total Loss: 0.27819 Val Src Loss: 0.24531 Val Tgt Loss: 0.03288
Test: 47 48


In [19]:
# sum(test_f1_list) / len(test_f1_list)

0.4307775450680297

In [20]:
f1_list = []

### 9-13 ###

In [21]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(9,14)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [22]:
test_loss, test_f1 = test(encoder, mlp, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 0.2805, Test F1: 0.6911


In [23]:
encoder_9_14, mlp_9_14, best_val_loss, best_val_src_loss, best_val_tgt_loss = continual_adapt([7,12,14], train_loader, val_loader, encoder, mlp, device, lambda_coeff=lambda_coeff, lr=lr)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 338.47it/s]


Epoch 1/500 Train Total Loss: 0.06514 Train Src Loss: 0.00839 Train Tgt Loss: 0.05675 
                 Val Total Loss: 0.27988 Val Src Loss: 0.20493 Val Tgt Loss: 0.07495


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 345.39it/s]


Epoch 2/500 Train Total Loss: 0.06056 Train Src Loss: 0.00836 Train Tgt Loss: 0.0522 
                 Val Total Loss: 0.2731 Val Src Loss: 0.1945 Val Tgt Loss: 0.0786


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 79.42it/s]


...

Epoch 13/500 Train Total Loss: 0.0462 Train Src Loss: 0.00707 Train Tgt Loss: 0.03912 
                 Val Total Loss: 0.27699 Val Src Loss: 0.21337 Val Tgt Loss: 0.06362


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 385.51it/s]


Epoch 14/500 Train Total Loss: 0.04191 Train Src Loss: 0.00697 Train Tgt Loss: 0.03494 
                 Val Total Loss: 0.28623 Val Src Loss: 0.22967 Val Tgt Loss: 0.05656


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 148.96it/s]

Epoch 15/500 Train Total Loss: 0.04433 Train Src Loss: 0.00668 Train Tgt Loss: 0.03764 
                 Val Total Loss: 0.28066 Val Src Loss: 0.22479 Val Tgt Loss: 0.05587


In [24]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Src Loss: {round(best_val_src_loss, 3)}, Val Tgt Loss: {round(best_val_tgt_loss, 3)}')

Total Val Loss: 0.269, Val Src Loss: 0.192, Val Tgt Loss: 0.078


### 14-18 ###

In [25]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(14,19)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [26]:
test_loss, test_f1 = test(encoder_9_14, mlp_9_14, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 0.4318, Test F1: 0.7207


In [27]:
encoder_14_19, mlp_14_19, best_val_loss, best_val_src_loss, best_val_tgt_loss = continual_adapt([12,17,19], train_loader, val_loader, encoder_9_14, mlp_9_14, device, lambda_coeff=lambda_coeff, lr=lr)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 466.84it/s]


Epoch 1/500 Train Total Loss: 0.099 Train Src Loss: 0.00827 Train Tgt Loss: 0.09073 
                 Val Total Loss: 0.29385 Val Src Loss: 0.21271 Val Tgt Loss: 0.08114


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 157.74it/s]


Epoch 2/500 Train Total Loss: 0.09651 Train Src Loss: 0.00755 Train Tgt Loss: 0.08896 
                 Val Total Loss: 0.28317 Val Src Loss: 0.21116 Val Tgt Loss: 0.07201


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 267.96it/s]


...

Epoch 17/500 Train Total Loss: 0.06473 Train Src Loss: 0.00943 Train Tgt Loss: 0.05529 
                 Val Total Loss: 0.2937 Val Src Loss: 0.23558 Val Tgt Loss: 0.05812


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 79.54it/s]


Epoch 18/500 Train Total Loss: 0.05525 Train Src Loss: 0.00819 Train Tgt Loss: 0.04706 
                 Val Total Loss: 0.30134 Val Src Loss: 0.24467 Val Tgt Loss: 0.05667


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 473.67it/s]

Epoch 19/500 Train Total Loss: 0.05844 Train Src Loss: 0.0093 Train Tgt Loss: 0.04914 
                 Val Total Loss: 0.31303 Val Src Loss: 0.25942 Val Tgt Loss: 0.05361


In [28]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Src Loss: {round(best_val_src_loss, 3)}, Val Tgt Loss: {round(best_val_tgt_loss, 3)}')

Total Val Loss: 0.27, Val Src Loss: 0.206, Val Tgt Loss: 0.064


### 19-23 ###

In [29]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(19,24)]
test_loader_19_24 = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [30]:
test_loss, test_f1 = test(encoder_14_19, mlp_14_19, test_loader_19_24, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 1.0233, Test F1: 0.4536


In [31]:
encoder_19_24, mlp_19_24, best_val_loss, best_val_src_loss, best_val_tgt_loss = continual_adapt([17,22,24], train_loader, val_loader, encoder_14_19, mlp_14_19, device, lambda_coeff=lambda_coeff, lr=lr)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 85.93it/s]


Epoch 1/500 Train Total Loss: 0.09946 Train Src Loss: 0.00789 Train Tgt Loss: 0.09157 
                 Val Total Loss: 0.29963 Val Src Loss: 0.21271 Val Tgt Loss: 0.08693


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 414.81it/s]


Epoch 2/500 Train Total Loss: 0.08505 Train Src Loss: 0.0078 Train Tgt Loss: 0.07725 
                 Val Total Loss: 0.3149 Val Src Loss: 0.22309 Val Tgt Loss: 0.09182


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 441.97it/s]


...

Epoch 19/500 Train Total Loss: 0.07715 Train Src Loss: 0.0075 Train Tgt Loss: 0.06965 
                 Val Total Loss: 0.30934 Val Src Loss: 0.2352 Val Tgt Loss: 0.07414


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 313.82it/s]


Epoch 20/500 Train Total Loss: 0.07526 Train Src Loss: 0.00894 Train Tgt Loss: 0.06632 
                 Val Total Loss: 0.31415 Val Src Loss: 0.23764 Val Tgt Loss: 0.07651


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 164.52it/s]

Epoch 21/500 Train Total Loss: 0.08272 Train Src Loss: 0.00938 Train Tgt Loss: 0.07334 
                 Val Total Loss: 0.30514 Val Src Loss: 0.23162 Val Tgt Loss: 0.07352


In [32]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Src Loss: {round(best_val_src_loss, 3)}, Val Tgt Loss: {round(best_val_tgt_loss, 3)}')

Total Val Loss: 0.281, Val Src Loss: 0.203, Val Tgt Loss: 0.078


### 24-28 ###

In [33]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(24,29)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [34]:
test_loss, test_f1 = test(encoder_19_24, mlp_19_24, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 1.7057, Test F1: 0.6111


In [35]:
encoder_24_29, mlp_24_29, best_val_loss, best_val_src_loss, best_val_tgt_loss = continual_adapt([22,27,29], train_loader, val_loader, encoder_19_24, mlp_19_24, device, lambda_coeff=lambda_coeff, lr=lr)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 414.74it/s]


Epoch 1/500 Train Total Loss: 0.08879 Train Src Loss: 0.00721 Train Tgt Loss: 0.08158 
                 Val Total Loss: 0.26805 Val Src Loss: 0.20286 Val Tgt Loss: 0.06519


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 382.19it/s]


Epoch 2/500 Train Total Loss: 0.08607 Train Src Loss: 0.00677 Train Tgt Loss: 0.07931 
                 Val Total Loss: 0.28507 Val Src Loss: 0.21657 Val Tgt Loss: 0.0685


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 194.71it/s]


...

Epoch 10/500 Train Total Loss: 0.07765 Train Src Loss: 0.00757 Train Tgt Loss: 0.07008 
                 Val Total Loss: 0.29523 Val Src Loss: 0.22669 Val Tgt Loss: 0.06854


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 463.08it/s]


Epoch 11/500 Train Total Loss: 0.08165 Train Src Loss: 0.00792 Train Tgt Loss: 0.07373 
                 Val Total Loss: 0.27367 Val Src Loss: 0.20551 Val Tgt Loss: 0.06815


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 479.08it/s]

Epoch 12/500 Train Total Loss: 0.07498 Train Src Loss: 0.00795 Train Tgt Loss: 0.06703 
                 Val Total Loss: 0.28915 Val Src Loss: 0.22175 Val Tgt Loss: 0.06741


In [36]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Src Loss: {round(best_val_src_loss, 3)}, Val Transfer Loss: {round(best_val_src_loss, 3)}')

Total Val Loss: 0.268, Val Src Loss: 0.203, Val Transfer Loss: 0.203


### 29-33 ###

In [37]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(29,34)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [38]:
test_loss, test_f1 = test(encoder_24_29, mlp_24_29, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 3.4053, Test F1: 0.3332


In [39]:
encoder_29_34, mlp_29_34, best_val_loss, best_val_src_loss, best_val_tgt_loss = continual_adapt([27,32,34], train_loader, val_loader, encoder_24_29, mlp_24_29, device, lambda_coeff=lambda_coeff, lr=lr)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 127.02it/s]


Epoch 1/500 Train Total Loss: 0.06796 Train Src Loss: 0.0068 Train Tgt Loss: 0.06117 
                 Val Total Loss: 0.28208 Val Src Loss: 0.21999 Val Tgt Loss: 0.06209


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 229.67it/s]


Epoch 2/500 Train Total Loss: 0.06055 Train Src Loss: 0.00705 Train Tgt Loss: 0.0535 
                 Val Total Loss: 0.28883 Val Src Loss: 0.21904 Val Tgt Loss: 0.06979


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 450.42it/s]


...

Epoch 24/500 Train Total Loss: 0.05266 Train Src Loss: 0.00829 Train Tgt Loss: 0.04438 
                 Val Total Loss: 0.30033 Val Src Loss: 0.25401 Val Tgt Loss: 0.04633


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 146.86it/s]


Epoch 25/500 Train Total Loss: 0.04922 Train Src Loss: 0.00876 Train Tgt Loss: 0.04046 
                 Val Total Loss: 0.28148 Val Src Loss: 0.23737 Val Tgt Loss: 0.04411


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 465.88it/s]

Epoch 26/500 Train Total Loss: 0.04937 Train Src Loss: 0.00822 Train Tgt Loss: 0.04115 
                 Val Total Loss: 0.26793 Val Src Loss: 0.22773 Val Tgt Loss: 0.0402


In [40]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Src Loss: {round(best_val_src_loss, 3)}, Val Tgt Loss: {round(best_val_tgt_loss, 3)}')

Total Val Loss: 0.267, Val Src Loss: 0.225, Val Tgt Loss: 0.042


### 34-38 ###

In [41]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(34,39)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [42]:
test_loss, test_f1 = test(encoder_29_34, mlp_29_34, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 4.0257, Test F1: 0.2796


In [43]:
encoder_34_39, mlp_34_39, best_val_loss, best_val_src_loss, best_val_tgt_loss = continual_adapt([32,37,39], train_loader, val_loader, encoder_29_34, mlp_29_34, device, lambda_coeff=lambda_coeff, lr=lr)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 461.85it/s]


Epoch 1/500 Train Total Loss: 0.04893 Train Src Loss: 0.00764 Train Tgt Loss: 0.04129 
                 Val Total Loss: 0.25764 Val Src Loss: 0.2277 Val Tgt Loss: 0.02994


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 158.03it/s]


Epoch 2/500 Train Total Loss: 0.0411 Train Src Loss: 0.00752 Train Tgt Loss: 0.03358 
                 Val Total Loss: 0.25072 Val Src Loss: 0.22727 Val Tgt Loss: 0.02345


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 197.07it/s]


...

Epoch 32/500 Train Total Loss: 0.03299 Train Src Loss: 0.00752 Train Tgt Loss: 0.02547 
                 Val Total Loss: 0.24724 Val Src Loss: 0.22751 Val Tgt Loss: 0.01973


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 467.70it/s]


Epoch 33/500 Train Total Loss: 0.03433 Train Src Loss: 0.00701 Train Tgt Loss: 0.02732 
                 Val Total Loss: 0.2612 Val Src Loss: 0.24473 Val Tgt Loss: 0.01646


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 88.88it/s]

Epoch 34/500 Train Total Loss: 0.03088 Train Src Loss: 0.00792 Train Tgt Loss: 0.02295 
                 Val Total Loss: 0.24908 Val Src Loss: 0.23394 Val Tgt Loss: 0.01514


In [44]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Src Loss: {round(best_val_src_loss, 3)}, Val Tgt Loss: {round(best_val_tgt_loss, 3)}')

Total Val Loss: 0.239, Val Src Loss: 0.224, Val Tgt Loss: 0.015


### 39-43 ###

In [45]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(39,44)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [46]:
test_loss, test_f1 = test(encoder_34_39, mlp_34_39, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 9.3898, Test F1: 0.1497


In [47]:
encoder_39_44, mlp_39_44, best_val_loss, best_val_src_loss, best_val_tgt_loss = continual_adapt([37,42,44], train_loader, val_loader, encoder_34_39, mlp_34_39, device, lambda_coeff=lambda_coeff, lr=lr)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 220.95it/s]


Epoch 1/500 Train Total Loss: 0.03118 Train Src Loss: 0.00757 Train Tgt Loss: 0.02361 
                 Val Total Loss: 0.26311 Val Src Loss: 0.2381 Val Tgt Loss: 0.02501


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 403.28it/s]


Epoch 2/500 Train Total Loss: 0.02608 Train Src Loss: 0.00667 Train Tgt Loss: 0.01941 
                 Val Total Loss: 0.25209 Val Src Loss: 0.22937 Val Tgt Loss: 0.02272


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 313.41it/s]


...

Epoch 17/500 Train Total Loss: 0.02761 Train Src Loss: 0.00806 Train Tgt Loss: 0.01956 
                 Val Total Loss: 0.25445 Val Src Loss: 0.23707 Val Tgt Loss: 0.01738


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 189.96it/s]


Epoch 18/500 Train Total Loss: 0.02552 Train Src Loss: 0.00789 Train Tgt Loss: 0.01764 
                 Val Total Loss: 0.25111 Val Src Loss: 0.23094 Val Tgt Loss: 0.02017


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 416.00it/s]

Epoch 19/500 Train Total Loss: 0.0253 Train Src Loss: 0.0074 Train Tgt Loss: 0.01791 
                 Val Total Loss: 0.26907 Val Src Loss: 0.24997 Val Tgt Loss: 0.0191


In [48]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Src Loss: {round(best_val_src_loss, 3)}, Val Tgt Loss: {round(best_val_tgt_loss, 3)}')

Total Val Loss: 0.24, Val Src Loss: 0.219, Val Tgt Loss: 0.021


### 44-48 ###

In [49]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(44,49)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [50]:
test_loss, test_f1 = test(encoder_39_44, mlp_39_44, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 6.2547, Test F1: 0.1576


In [51]:
sum(f1_list) / len(f1_list)

0.4245600752170377